In [1]:
# 实现交互式图表
%matplotlib qt5

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne

# Loading data

In [3]:
data_folder = 'raw_data';
items = os.listdir(data_folder)#列出目录中的文件名/子文件夹
dataList = []
for filenames in items:
    if filenames.endswith(".set"):
        dataList.append(filenames)

In [ ]:
# 另一种写法（需要遍历子目录时）：
# dataList = []
# for parent, dirnames, filenames in os.walk(data_folder):
#     for filename in filenames:
#         if os.path.splitext(filename)[1] == '.set':
#             dataList.append(filename)
#     break #若去掉break则可以深层次遍历所有子目录

In [4]:
dataFile = dataList[0]

In [5]:
data_file = os.path.join(data_folder,dataFile)
data = mne.io.read_raw_eeglab(data_file, eog = 'auto') #另一个mne.read_epochs_eeglab()读取已经分段的数据
# 当然也可以后面更改通道类型
# data.set_channel_types({'EOG': 'eog'})

Reading C:\Users\Surface\Desktop\MNE\My_scripts\N170_data\raw_data\n170_data.fdt


<ipython-input-5-2e29decf5359>:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  data = mne.io.read_raw_eeglab(data_file, eog = 'auto') #另一个mne.read_epochs_eeglab()读取已经分段的数据
<ipython-input-5-2e29decf5359>:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  data = mne.io.read_raw_eeglab(data_file, eog = 'auto') #另一个mne.read_epochs_eeglab()读取已经分段的数据


# Browsing data

In [6]:
fig = data.plot(duration=5, n_channels=len(data.ch_names),scalings=dict(eeg=80e-6))#scaling 80uV
# fig.canvas.key_press_event('a')
# 通过添加bad annotation手动识别伪迹span

Channels marked as bad: none


# Channel location

In [ ]:
"""
电极位置配置：读取电极坐标文件
坐标位置在data.info的chs和dig
"""
montage = mne.channels.read_custom_montage('standard-10-5-cap385.elp')
data.set_montage(montage)

In [ ]:
"""
电极位置配置：MNE内置模板
"""
montage = mne.channels.make_standard_montage('standard_1020')
data.set_montage(montage)
# 或合并写
# data_1020 = data.copy().set_montage('standard_1020')

In [ ]:
# 绘制电极位置
# data.plot_sensors()
# plt.show()
# 或同时画2D和3D
# fig = plt.figure()
# ax2d = fig.add_subplot(121)
# ax3d = fig.add_subplot(122, projection='3d')
# data.plot_sensors(ch_type='eeg', axes=ax2d)
# data.plot_sensors(ch_type='eeg', axes=ax3d, kind='3d')
# ax3d.view_init(azim=70, elev=15)

In [ ]:
"""
绘制通道位置图，并对应位置上显示通道名称
"""
# layout_from_data = mne.channels.make_eeg_layout(data.info)
# #或 layout_from_data =mne.channels.find_layout(data.info, ch_type='eeg')
# layout_from_data.plot()
# plt.show()

# Bad channels

1. 标记坏导

In [ ]:
from copy import deepcopy

# original_bads = deepcopy(data.info['bads'])
data.info['bads'].append('Fpz')               # add a single channel
# OR
# data.info['bads'] = ['MEG 2443']
# data.info['bads'].extend(['Fpz', 'Oz'])  # add a list of channels
# bad_chan = data.info['bads'].pop(-1)  # remove the last entry in the list
# data.info['bads'] = original_bads     # change the whole list at once

# 或通过data.plot画图直接在图上点击坏导，数据‘bads’会自动更新坏导信息
# data.plot(block=True)#pause the script while the plot is open, giving you time to mark bad channels
# plt.show()

In [ ]:
# data2 = data.copy()
# data2.info['bads'] = []
# events = mne.events_from_annotations(data2)
# epochs = mne.Epochs(data2, events = events)['2'].average().plot()

2. Interpolation（插值法）：spherical spline method

In [ ]:
data.load_data()
eeg_data = data.pick_types(eeg=True, eog=False, exclude=[])
eeg_data_interp = eeg_data.interpolate_bads()#reset_bads : If True, remove the bads from info.

# Resampling

In [ ]:
data_downsampled = eeg_data_interp.resample(sfreq=250)

# Filtering

1. Highpass/Lowpass/Bandpass filter

In [ ]:
# data_highpass = data_downsampled.filter(l_freq=0.1, h_freq=None)#0.1Hz高通，去除慢波漂移伪迹
# data_lowpass = data_downsampled.filter(h_freq=35, l_freq=None)#35Hz低通
data_bandpass = data_downsampled.filter(l_freq=0.1, h_freq=35)

2. Notch filter

In [ ]:
data_notch = data_bandpass.notch_filter(freqs=50)

# Reference

In [ ]:
# use a single channel reference (left earlobe or others)
# data_notch.set_eeg_reference(ref_channels=['A1'])

# use average of mastoid channels as reference
data_mastoid_ref = data_notch.set_eeg_reference(ref_channels=['M1', 'M2'])#默认average，还可选'REST'
# data_avg_ref = data_notch.set_eeg_reference(ref_channels='average',projection=True)
'''
If ``projection=True``, the average reference is
    added as a projection and is not applied to the data (it can be
    applied afterwards with the ``apply_proj`` method). If
    ``projection=False``, the average reference is directly applied to
    the data. If ``ref_channels`` is not ``'average'``, ``projection``
    must be set to ``False``
'''

若要先恢复线上参考电极

In [ ]:
# add new reference channel (all zero)
data_new_ref = mne.add_reference_channels(data_notch, ref_channels=['CPz'])#默认copy，而不改变raw
# set reference to `xxx`
data_mastoid_ref = data_new_ref.set_eeg_reference(ref_channels=['M1', 'M2'])

# ICA

In [ ]:
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)

## 1. Fitting and plotting the ICA solution

In [ ]:
ica = ICA(n_components=60, max_pca_components=60,random_state=97)
ica.fit(data_mastoid_ref)

In [ ]:
data.load_data()
ica.plot_sources(data)
plt.show()

In [ ]:
ica.plot_components(inst=data)
plt.show()

In [ ]:
# blinks
ica.plot_overlay(data, exclude=[1], picks='eeg')

In [ ]:
ica.plot_properties(data, picks=[0, 1])

## Selecting ICA components manually

In [ ]:
ica.exclude = [2]  # indices chosen based on various plots above

# ica.apply() changes the Raw object in-place, so let's make a copy first:
reconst_data = data_mastoid_ref.copy()
ica.apply(reconst_data)

In [ ]:
# 对比去眼电前后
data_mastoid_ref.plot()
event_dict.plot()
plt.show()

# Events

In [ ]:
events, event_id = mne.events_from_annotations(data)

In [ ]:
events_pick = mne.pick_events(events, exclude=[1,6,7,8])

Mapping Event IDs to trial descriptors

In [ ]:
event_dict = {'face/up': 2, 'chair/up': 3, 'face/down': 4,'chair/down': 5}

# Epoch

epoch数据结构：NumPy array (n_epochs, n_channels, n_times)

## 1. Creating Epoched data

In [ ]:
# epochs = mne.Epochs(reconst_data, events_pick, event_id=event_dict, 
#                     tmin=-0.2, tmax=0.8, preload=True)#baseline correction was automatically applied 

In [ ]:
reject.update({'eog': 150e-6})
epochs.drop_bad(reject=reject)# Rejecting Epochs based on channel amplitude
reject_criteria = dict(eeg=150e-6)#can play around with this
flat_criteria = dict(eeg=1e-6)
epochs = mne.Epochs(reconst_data, events_pick, event_id=event_dict,
                    tmin=-0.2, tmax=0.8, 
                    reject=reject_criteria, flat=flat_criteria,
                    reject_by_annotation=False, preload=True)
# epochs.plot_drop_log()
# plt.show()

In [ ]:
# # Reject bad epochs during configuration
# reject = dict(eeg=100e-6)
# epochs = mne.Epochs(data_mastoid_ref, events_pick, event_id=event_dict, reject=reject, 
#                     tmin=-0.2, tmax=0.8, preload=True)#baseline correction was automatically applied 

# # You can also reject after constructing epochs
# reject.update({'eeg': 80e-6})
# epochs.drop_bad(reject=reject)

# Check & reject epoch manually
# fig = epochs.plot(picks=['eeg'], scalings=dict(eeg=100e-6), n_epochs=2, n_channels=63, block=True)
# plt.show()

## Autoreject

Autoreject (global) can compute the rejection dictionary automatically

In [ ]:
from autoreject import get_rejection_threshold  # noqa
reject = get_rejection_threshold(epochs)
print(reject)

## 2. Subselecting epochs

In [ ]:
face_epoch = epochs['face']
chair_epoch = epochs['chair']

In [ ]:
#Selecting epochs by index
# print(epochs[:10])    # epochs 0-9
# print(epochs[1:8:2])  # epochs 1, 3, 5, 7

# print(epochs['face'][:4])            # first 4 "face" epochs
# print(epochs['face'][[0, 1, 2, 3]])  # same as previous line

# 截取epoch
# shorter_epochs = epochs.copy.crop(tmin=-0.1, tmax=0.1, include_tmax=True)

In [ ]:
# Converting an Epochs object to a DataFrame
df = epochs.to_data_frame()
df.iloc[:10, :10]

In [ ]:
# Scaling time and channel values
df = epochs.to_data_frame(time_format=None,
                          scalings=dict(eeg=1, mag=1, grad=1))
df.iloc[:5, :10]

In [ ]:
# Setting the index
df = epochs.to_data_frame(index=['condition', 'epoch'],
                          time_format=None)
df.iloc[:5, :10]

In [ ]:
long_df = epochs.to_data_frame(time_format=None, index='condition',
                               long_format=True)
long_df.head()

In [ ]:
import seaborn as sns

channels = ['P8', 'Fp1']
data = long_df.loc['face/up'].query('channel in @channels')
# convert channel column (CategoryDtype → string; for a nicer-looking legend)
data['channel'] = data['channel'].astype(str)
sns.lineplot(x='time', y='value', hue='channel', data=data)

In [ ]:
# saving:文件名必须以-epo or _epo结尾
epochs.save('clean-epoch-epo.fif', overwrite=True)

# ERP Analysis

In [ ]:
epochs.average() #返回evoke，值为平均
epochs.standard_error()#返回evoke，值为std